# Project 6 Custom EDA
Beth Spornitz
September 23, 2024

This code uses a combination of Python and Markdown to create an initial data story in a Jupyter Notebook. The project includes a project virtual environment with popular libraries for data analytics including pandas, matplotlib, and seaborn, and introduces a common process for starting exploratory data analysis projects.

Specific to the titanic dataset, this file will walk through how to perform these functions:
1.  Importing Requirements
2.  Data Acquisition
3.  Inital Data Inspection
4.  Showing initial descriptive statistics
5.  Showing initial data distribution for both categorical and numerical columns
6.  Data Cleaning
7.  Data Transformation

These steps will be followed by visulations and explanations of visualizations.

### Import Dependencies

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

### Import Local Modules

In [ ]:
import utils_bethspornitz
import bethspornitz_project_setup

### Data Acquisition

In [ ]:
# Load the Iris dataset into DataFrame
df = sns.load_dataset('titanic')

#Read the file
pd.read_csv('titanic.csv')

# Inspect first rows of the DataFrame
print(df.head())